In [1]:
%pip install arcgis


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import requests
import json
import webbrowser
from arcgis.gis import GIS
from getpass import getpass

In [3]:
pwd = getpass()
gis = GIS(username="thabo.traore_AAMU", password=pwd)
token = gis._con.token

In [4]:
survey_api_url = "https://survey123.arcgis.com/api/surveys"
portalUrl = "https://aamu.maps.arcgis.com"

In [5]:
search = "search"

payload = {
    "searchString": "City of Rocky Mount Service Line Material Survey",
    "searchSurveyType": "owned",
    "isPublished": "True",
    "token": token,
    "portalUrl": portalUrl
}

request_url = '{0}{1}'.format(survey_api_url, search)
r = requests.get(request_url, params=payload)

if r.status_code == requests.codes.ok:
    print("...Results count: ", json.loads(r.content))
    for search_result in json.loads(r.content):
        print("\n...Search result: \n", search_result)
else:
    r.raise_for_status()

...Results count:  {'errorCode': 101, 'errorMessage': 'Unknown service'}

...Search result: 
 errorCode

...Search result: 
 errorMessage


In [6]:
%pip install requests pandas reportlab

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [25]:
import requests
import pandas as pd
from reportlab.lib.pagesizes import letter
from reportlab.pdfgen import canvas

# Replace these with your ArcGIS details
FEATURE_LAYER_URL = "https://services5.arcgis.com/s1DFLqWfkSIcXZ9O/ArcGIS/rest/services/survey123_24aa1572e50e42e8aa193dad894f57af_results/FeatureServer/0/query"
TOKEN = "mzFcMRqhxzPAoRJavp2MJg-w3Fau_D0Ix0Qviif4Gs2B6QMLbfdRhemy-rS5IGGgZCF2xD3K_WcpdZ8nn-17hTnUOnEP5RL_cjeIGNGhJGG1jD8yRh774A5PlAhJTqUuF6Owkwz3p-NTZzOMCUy8Klk4bkKVvGMn16fghXnSHGoe0PpukZVRxTAYz9K2dNkG"

# Function to fetch data from the feature layer
def fetch_arcgis_data():
    params = {
        "where": "1=1",  # Get all records
        "outFields": "*",  # Fetch all fields
        "f": "json",  # Format response as JSON
        "token": TOKEN  # Authentication token
    }

    response = requests.get(FEATURE_LAYER_URL, params=params)
    
    if response.status_code == 200:
        data = response.json()
        features = data.get("features", [])
        return [feature["attributes"] for feature in features]
    else:
        print("Error fetching data:", response.text)
        return []

# Function to create a PDF report
def generate_pdf_report(data, filename="ArcGIS_Report_lead_service_line.pdf"):
    c = canvas.Canvas(filename, pagesize=letter)
    c.setFont("Helvetica", 12)
    
    c.drawString(100, 750, "ArcGIS Feature Layer Report")
    c.drawString(100, 730, f"Total Records: {len(data)}")
    
    y_position = 700
    for index, row in enumerate(data[:20]):  # Show only first 20 rows
        c.drawString(100, y_position, f"Record {index + 1}: {row}")
        y_position -= 20
        if y_position < 50:  # Create a new page if needed
            c.showPage()
            y_position = 750

    c.save()
    print(f"PDF report saved as {filename}")

# Main execution
if __name__ == "__main__":
    records = fetch_arcgis_data()
    if records:
        df = pd.DataFrame(records)
        print(df.head())  # Display some data
        generate_pdf_report(records)
    else:
        print("No data retrieved.")


   objectid                              globalid   CreationDate  \
0         1  39f786ae-8930-4d8e-8cc4-5e58432cdaad  1741816072781   
1         2  1fdabcd0-bf53-450d-8dd0-445a69792eb7  1743613816696   

             Creator       EditDate             Editor      staff_name  \
0  thabo.traore_AAMU  1741816072781  thabo.traore_AAMU  Sarah Mitchell   
1  thabo.traore_AAMU  1743613816696  thabo.traore_AAMU         Ibrahim   

   field_survey_date service_type              address system_owned_material  \
0      1741798800000     Potholed    123 Oak Dr, Rocky                Copper   
1      1743613200000     Potholed  Huntsville, AL, USA                Copper   

  system_owned_material_other  \
0                        None   
1                        None   

                                  system_owned_notes customer_owned_material  \
0  The system is currently working with minimal i...                    Lead   
1                                                N/A                  C

In [1]:
import requests
import pandas as pd
from reportlab.lib.pagesizes import letter, landscape
from reportlab.lib import colors
from reportlab.platypus import SimpleDocTemplate, Table, TableStyle

# Replace these with your ArcGIS details
FEATURE_LAYER_URL = "https://services5.arcgis.com/s1DFLqWfkSIcXZ9O/ArcGIS/rest/services/survey123_24aa1572e50e42e8aa193dad894f57af_results/FeatureServer/0/query"
TOKEN = "mzFcMRqhxzPAoRJavp2MJg-w3Fau_D0Ix0Qviif4Gs2B6QMLbfdRhemy-rS5IGGgZCF2xD3K_WcpdZ8nn-17hTnUOnEP5RL_cjeIGNGhJGG1jD8yRh774A5PlAhJTqUuF6Owkwz3p-NTZzOMCUy8Klk4bkKVvGMn16fghXnSHGoe0PpukZVRxTAYz9K2dNkG"

# Function to fetch data from the feature layer
def fetch_arcgis_data():
    params = {
        "where": "1=1",  # Get all records
        "outFields": "*",  # Fetch all fields
        "f": "json",  # Format response as JSON
        "token": TOKEN  # Authentication token
    }

    try:
        response = requests.get(FEATURE_LAYER_URL, params=params, timeout=15)
        response.raise_for_status()  # Raise HTTP error if one occurs
        data = response.json()

        # Check for ArcGIS API errors
        if "error" in data:
            print("ArcGIS API Error:", data["error"])
            return []

        features = data.get("features", [])
        return [feature["attributes"] for feature in features]

    except requests.exceptions.RequestException as e:
        print("Request error:", e)
        return []

# Function to create a well-formatted PDF report
def generate_pdf_report(data, filename="ArcGIS_Report.pdf"):
    if not data:
        print("No data available for the report.")
        return

    # Convert data to a Pandas DataFrame for better handling
    df = pd.DataFrame(data)

    # Define columns to include in the report
    columns_to_include = [
        "staff_name", "field_survey_date", "service_type", "address",
        "system_owned_material", "system_owned_notes", "customer_owned_material",
        "lead_fittings", "fittingssolder_notes"
    ]

    # Rename columns to match your dataset headers
    column_headers = {
        "staff_name": "Staff Name",
        "field_survey_date": "Field Survey Date",
        "service_type": "Service Type",
        "address": "Address",
        "system_owned_material": "System Owned Material",
        "system_owned_notes": "System Owned Notes",
        "customer_owned_material": "Customer Owned Material",
        "lead_fittings": "Lead Fittings",
        "fittingssolder_notes": "Fittings/Solder Notes",
        "photo": "Photo",
        "files_and_other_documents": "Files and Other Documents"
    }

    # Keep only selected columns
    df = df[columns_to_include]
    df.rename(columns=column_headers, inplace=True)

    # Convert DataFrame to list format for the report
    table_data = [df.columns.tolist()] + df.values.tolist()

    # Create PDF document (Landscape for better table fit)
    pdf = SimpleDocTemplate(filename, pagesize=landscape(letter))
    table = Table(table_data, repeatRows=1)

    # Add styling
    style = TableStyle([
        ('BACKGROUND', (0, 0), (-1, 0), colors.grey),
        ('TEXTCOLOR', (0, 0), (-1, 0), colors.whitesmoke),
        ('ALIGN', (0, 0), (-1, -1), 'LEFT'),
        ('FONTNAME', (0, 0), (-1, 0), 'Helvetica-Bold'),
        ('BOTTOMPADDING', (0, 0), (-1, 0), 12),
        ('BACKGROUND', (0, 1), (-1, -1), colors.beige),
        ('GRID', (0, 0), (-1, -1), 1, colors.black),
        ('WORDWRAP', (0, 0), (-1, -1)),  # Wrap text if too long
    ])
    table.setStyle(style)

    # Build and save the PDF
    pdf.build([table])
    print(f"✅ PDF report saved as {filename}")

# Main execution
if __name__ == "__main__":
    records = fetch_arcgis_data()
    if records:
        df = pd.DataFrame(records)
        print(df.head())  # Display sample data in console
        generate_pdf_report(records)
    else:
        print("No data retrieved.")


ArcGIS API Error: {'code': 498, 'message': 'Invalid token.', 'details': ['Invalid token.']}
No data retrieved.


In [27]:
import requests

# Base info
FEATURE_LAYER_URL = "https://services5.arcgis.com/s1DFLqWfkSIcXZ9O/ArcGIS/rest/services/survey123_24aa1572e50e42e8aa193dad894f57af_results/FeatureServer/0"
TOKEN = "mzFcMRqhxzPAoRJavp2MJg-w3Fau_D0Ix0Qviif4Gs2B6QMLbfdRhemy-rS5IGGgZCF2xD3K_WcpdZ8nn-17hTnUOnEP5RL_cjeIGNGhJGG1jD8yRh774A5PlAhJTqUuF6Owkwz3p-NTZzOMCUy8Klk4bkKVvGMn16fghXnSHGoe0PpukZVRxTAYz9K2dNkG"

# Step 1: Get all OBJECTIDs
def get_object_ids():
    url = f"{FEATURE_LAYER_URL}/query"
    params = {
        "where": "1=1",
        "returnIdsOnly": "true",
        "f": "json",
        "token": TOKEN
    }
    r = requests.get(url, params=params)
    if r.status_code == 200:
        return r.json().get("objectIds", [])
    else:
        print("Error getting object IDs:", r.text)
        return []

# Step 2: Check for attachments
def get_attachments_for_object(object_id):
    url = f"{FEATURE_LAYER_URL}/{object_id}/attachments"
    params = {
        "f": "json",
        "token": TOKEN
    }
    r = requests.get(url, params=params)
    if r.status_code == 200:
        return r.json().get("attachmentInfos", [])
    return []

# Step 3: Loop and find those with images
def list_image_urls():
    object_ids = get_object_ids()
    all_image_links = []
    
    for oid in object_ids:
        attachments = get_attachments_for_object(oid)
        for att in attachments:
            image_url = f"{FEATURE_LAYER_URL}/{oid}/attachments/{att['id']}?token={TOKEN}"
            all_image_links.append({
                "object_id": oid,
                "filename": att["name"],
                "url": image_url
            })

    return all_image_links

# Main
if __name__ == "__main__":
    images = list_image_urls()
    if images:
        print("📸 Found image attachments:")
        for img in images:
            print(f"OID {img['object_id']}: {img['filename']} → {img['url']}")
    else:
        print("No image attachments found.")


📸 Found image attachments:
OID 1: ff49ad65d47b9a16bb587d7ac1959551.jpg → https://services5.arcgis.com/s1DFLqWfkSIcXZ9O/ArcGIS/rest/services/survey123_24aa1572e50e42e8aa193dad894f57af_results/FeatureServer/0/1/attachments/1?token=mzFcMRqhxzPAoRJavp2MJg-w3Fau_D0Ix0Qviif4Gs2B6QMLbfdRhemy-rS5IGGgZCF2xD3K_WcpdZ8nn-17hTnUOnEP5RL_cjeIGNGhJGG1jD8yRh774A5PlAhJTqUuF6Owkwz3p-NTZzOMCUy8Klk4bkKVvGMn16fghXnSHGoe0PpukZVRxTAYz9K2dNkG
OID 2: 1_Inch_Inside_SCH_40_PVC_Pipe.jpg → https://services5.arcgis.com/s1DFLqWfkSIcXZ9O/ArcGIS/rest/services/survey123_24aa1572e50e42e8aa193dad894f57af_results/FeatureServer/0/2/attachments/2?token=mzFcMRqhxzPAoRJavp2MJg-w3Fau_D0Ix0Qviif4Gs2B6QMLbfdRhemy-rS5IGGgZCF2xD3K_WcpdZ8nn-17hTnUOnEP5RL_cjeIGNGhJGG1jD8yRh774A5PlAhJTqUuF6Owkwz3p-NTZzOMCUy8Klk4bkKVvGMn16fghXnSHGoe0PpukZVRxTAYz9K2dNkG
OID 2: Ibrahim Traore_Official Excused Absence_ Rocky Mount Site Visit.pdf → https://services5.arcgis.com/s1DFLqWfkSIcXZ9O/ArcGIS/rest/services/survey123_24aa1572e50e42e8aa193dad894f57af

In [28]:
import os
import requests

# Found image links from earlier
attachments = [
    {
        "object_id": 1,
        "filename": "ff49ad65d47b9a16bb587d7ac1959551.jpg",
        "url": "https://services5.arcgis.com/s1DFLqWfkSIcXZ9O/ArcGIS/rest/services/survey123_24aa1572e50e42e8aa193dad894f57af_results/FeatureServer/0/1/attachments/1?token=mzFcMRqhxzPAoRJavp2MJg-w3Fau_D0Ix0Qviif4Gs2B6QMLbfdRhemy-rS5IGGgZCF2xD3K_WcpdZ8nn-17hTnUOnEP5RL_cjeIGNGhJGG1jD8yRh774A5PlAhJTqUuF6Owkwz3p-NTZzOMCUy8Klk4bkKVvGMn16fghXnSHGoe0PpukZVRxTAYz9K2dNkG"
    },
    {
        "object_id": 2,
        "filename": "1_Inch_Inside_SCH_40_PVC_Pipe.jpg",
        "url": "https://services5.arcgis.com/s1DFLqWfkSIcXZ9O/ArcGIS/rest/services/survey123_24aa1572e50e42e8aa193dad894f57af_results/FeatureServer/0/2/attachments/2?token=mzFcMRqhxzPAoRJavp2MJg-w3Fau_D0Ix0Qviif4Gs2B6QMLbfdRhemy-rS5IGGgZCF2xD3K_WcpdZ8nn-17hTnUOnEP5RL_cjeIGNGhJGG1jD8yRh774A5PlAhJTqUuF6Owkwz3p-NTZzOMCUy8Klk4bkKVvGMn16fghXnSHGoe0PpukZVRxTAYz9K2dNkG"
    },
    {
        "object_id": 2,
        "filename": "Ibrahim Traore_Official Excused Absence_ Rocky Mount Site Visit.pdf",
        "url": "https://services5.arcgis.com/s1DFLqWfkSIcXZ9O/ArcGIS/rest/services/survey123_24aa1572e50e42e8aa193dad894f57af_results/FeatureServer/0/2/attachments/3?token=mzFcMRqhxzPAoRJavp2MJg-w3Fau_D0Ix0Qviif4Gs2B6QMLbfdRhemy-rS5IGGgZCF2xD3K_WcpdZ8nn-17hTnUOnEP5RL_cjeIGNGhJGG1jD8yRh774A5PlAhJTqUuF6Owkwz3p-NTZzOMCUy8Klk4bkKVvGMn16fghXnSHGoe0PpukZVRxTAYz9K2dNkG"
    }
]

# Create download folder if it doesn't exist
download_folder = "attachments"
os.makedirs(download_folder, exist_ok=True)

# Download each file
for att in attachments:
    object_id = att["object_id"]
    filename = att["filename"]
    url = att["url"]

    safe_filename = f"{object_id}_{filename.replace(' ', '_')}"
    file_path = os.path.join(download_folder, safe_filename)

    print(f"⬇️ Downloading: {filename} → {file_path}")
    try:
        response = requests.get(url)
        if response.status_code == 200:
            with open(file_path, "wb") as f:
                f.write(response.content)
            print("✅ Downloaded.")
        else:
            print(f"❌ Failed to download {filename}: {response.status_code}")
    except Exception as e:
        print(f"❌ Error downloading {filename}: {e}")


⬇️ Downloading: ff49ad65d47b9a16bb587d7ac1959551.jpg → attachments\1_ff49ad65d47b9a16bb587d7ac1959551.jpg
✅ Downloaded.
⬇️ Downloading: 1_Inch_Inside_SCH_40_PVC_Pipe.jpg → attachments\2_1_Inch_Inside_SCH_40_PVC_Pipe.jpg
✅ Downloaded.
⬇️ Downloading: Ibrahim Traore_Official Excused Absence_ Rocky Mount Site Visit.pdf → attachments\2_Ibrahim_Traore_Official_Excused_Absence__Rocky_Mount_Site_Visit.pdf
✅ Downloaded.
